In [2]:
import numpy as np

In [3]:
def read_input(infile):
    data = []
    with open(infile, 'r') as inf:
        for line in inf.readlines():
            data.append([-1 if c == '#' else 0 if c == '.' else 2 if c == 'S' else 3 for c in line.strip()])
    return np.array(data)

In [68]:
def find_path(data):
    # print(data)
    sy, sx = np.argwhere(data==2)[0]
    ty, tx = np.argwhere(data==3)[0]

    cost_u = np.copy(data)
    cost_u[sy, sx] = 0
    cost_u[ty, tx] = 0
    cost_u[np.where(cost_u==0)] = 1e9

    # Store the cost to reach this square and look into a given direction
    costs = {(-1,0): cost_u, 
             (0,1): np.copy(cost_u),
             (1,0): np.copy(cost_u), 
             (0,-1): np.copy(cost_u)}

    pos = [(sy,sx,0,1,0)]

    while len(pos) != 0:

        y, x, dy, dx, score = pos.pop(0)

        for ndy, ndx in [(0, 1), (-1, 0), (0, -1), (1, 0)]:

            ny, nx = y+ndy, x+ndx

            if data[ny, nx] < 0:
                # Wall
                continue

            cost_turn = 1000 * max(abs(ndy-dy), abs(ndx-dx))

            if cost_turn > 1000:
                # No turning back
                continue

            # If we turn is the score for _this_ square better than what we
            # got so far to look into that direction?
            if costs[(ndy, ndx)][y,x] > cost_turn + score:
                costs[(ndy, ndx)][y,x] = cost_turn + score
                # Update the cost map, and add this pos as a new point to check
                pos.append((y, x, ndy, ndx, cost_turn+score))

            # Is it a good idea to move in this direction?
            if costs[(ndy, ndx)][ny,nx] > cost_turn + score + 1:
                # Move in that direction
                costs[(ndy, ndx)][ny,nx] = cost_turn + score + 1
                pos.append((ny, nx, ndy, ndx, cost_turn+score+1))

        pos.sort(key=lambda tup: tup[4])

    # Return the 4 cost maps
    return costs

def lowest_cost(data):
    # Check the cost maps for the lowest cost at the end
    costs = find_path(data)
    ty, tx = np.argwhere(data==3)[0]
    best = 1e9
    for c in costs.values():
        best = min(c[ty, tx], best)

    return int(best)

def best_seats(data):
    # Starting from the end, backtrack the possible paths leading here
    costs = find_path(data)
    ty, tx = np.argwhere(data==3)[0]
    sy, sx = np.argwhere(data==2)[0]

    best = 1e9
    for c in costs.values():
        best = min(c[ty, tx], best)

    pos = []
    path = []
    seats = 0
    for k, c in costs.items():
        if c[ty,tx] == best:
            if (ty, tx) not in path:
                path.append((ty, tx))
                pos.append((ty, tx, k[0], k[1]))

    while len(pos) != 0:
        y, x, dy, dx = pos.pop(0)

        if (y,x) == (sy, sx):
            seats += 1
            continue

        cost = costs[(dy, dx)][y, x]

        for k, c in costs.items():
            ny = y-k[0]
            nx = x-k[1]

            if c[ny,nx] == cost - 1 and (dy, dx) == k:
                if (ny, nx) not in path:
                    path.append((ny, nx))
                    pos.append((ny, nx, k[0], k[1]))
                    seats += 1
            elif c[ny,nx] == cost - 1001:
                if (ny, nx) not in path:
                    path.append((ny, nx))
                    pos.append((ny, nx, k[0], k[1]))
                    seats += 1

    return seats

In [69]:
print('*******\nPuzzle1\n*******\n')

print('Test case\n---------\n')

res = lowest_cost(read_input('input16a.txt'))

print(f'Score is {res}')

assert res == 7036

print('\nPuzzle case\n-----------\n')

res = lowest_cost(read_input('input16.txt'))

print(f'Score is {res}')

assert res == 109496

print('\n*******\nPuzzle2\n*******\n')

print('Test case\n---------\n')

res = best_seats(read_input('input16a.txt'))

print(f'Number of tiles is {res}')

assert res == 45

print('\nPuzzle case\n-----------\n')

res = best_seats(read_input('input16.txt'))

print(f'Number of tiles is {res}')

assert res == 551


*******
Puzzle1
*******

Test case
---------

Score is 7036

Puzzle case
-----------

Score is 109496

*******
Puzzle2
*******

Test case
---------

Number of tiles is 45

Puzzle case
-----------

Number of tiles is 551
